In [1]:
import os
import time
import platform
import datetime
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver as wd
from IPython.display import clear_output

In [2]:
chromedriver ='c:/chromedriver.exe'

if platform.system() == 'Darwin': 
    driver = wd.Chrome(chromedriver)
    driver.implicitly_wait(2)
else :    
    driver = wd.Chrome(chromedriver)
    driver.implicitly_wait(2)

In [3]:
URL = 'https://www.safekorea.go.kr/idsiSFK//neo/sfk/cs/sfc/acd/iflccwUserList.jsp?menuSeq=99'
driver.get(URL)

driver.find_element_by_id('q_startDt').clear()
driver.find_element_by_id('q_endDt').clear()

In [4]:
driver.find_element_by_id('q_startDt').send_keys(input('Input start_date : '))
driver.find_element_by_id('q_endDt').send_keys(input('Input end date : '))
driver.find_element_by_xpath('//*[@id="content"]/div[2]/a').click()

Input start_date : 20180101
Input end date : 20210729


In [5]:
def getData() :

    data_dict = {
                '지역'     : [],
                '발생일시' : [],
                '복구일시' : [],
                '정전가구' : []
                }
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    min_page = soup.select_one('#minPage').text
    max_page = soup.select_one('#maxPage').text

    a1   = soup.select_one('#content > div.boardList_boxWrap > table')

    # 컬렁명 코드.
    # for i in a1.select('thead > tr > th' ) :
    #     title = i.text.strip()
    #     data_dict['제목'] += [title]

    a1.select_one('#apiTr_0_dt')
    for idx, j in enumerate(range(10)) :
        a2 = a1.select(f'#apiTr_{j}_apiData1')
        for j in a2 :
            s = j.text.strip().split('\n')
            data_dict['지역'].append(s[0])
            data_dict['발생일시'].append(s[1].split('/')[0])
            data_dict['복구일시'].append(s[1].split('/')[-1])
            data_dict['정전가구'].append(s[-1])
            # print(s)
    
    df = pd.DataFrame(data_dict)
    if os.path.exists('data_정전.csv') == False :
        df.to_csv('data_정전.csv', encoding='utf-8-sig', index=False)
    else :
        df.to_csv('data_정전.csv', encoding='utf-8-sig', index=False, header=False, mode='a')

    return min_page, max_page

Start = time.time()

while True :

    time.sleep(1)
    clear_output(wait=True)
    min_page, max_page = getData()
    print( f'{min_page}/{max_page} Page 수집중')
    if min_page != max_page : 
        driver.find_element_by_xpath('//*[@id="content"]/div[4]/div/div/a[2]').click()
    else : break

End = time.time()   
print(f'수집완료({datetime.timedelta(seconds = End - Start)})')        

246/246 Page 수집중
수집완료(0:04:24.202283)


In [6]:
driver.close()
driver.quit()

In [8]:
df = pd.read_csv('data_정전.csv')
df

,지역,발생일시,복구일시,정전가구
0,충청북도 음성군,2021.07.28 14:20,2021.07.28 09:25,20000
1,충청북도 음성군,2021.07.27 14:15,2021.07.27 09:59,0
2,충청남도 논산시,2021.07.22 15:06,2021.07.22 15:04,2000
3,충청남도 논산시,2021.07.22 14:50,2021.07.22 14:55,2000
4,부산광역시 남구,2021.07.22 14:36,2021.07.22 10:36,1000
...,...,...,...,...
2454,전라남도 광양시,2018.07.03 10:34,2018.07.03 11:10,1
2455,전라남도 광양시,2018.07.03 07:51,2018.07.03 08:10,1
2456,전라남도 광양시,2018.07.03 07:36,2018.07.03 08:00,1
2457,전라남도 구례군,2018.07.02 09:00,2018.07.02 09:32,249
